In [85]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [110]:
df=pd.read_excel('clemson_student.xlsx',engine='openpyxl')

In [111]:
df.columns=[str.lower('_'.join(col.split())) for col in df.columns]

In [193]:
# df.info()

In [89]:
df.shape

(89, 30)

<h1>R1 R2</h1>

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 30 columns):
 #   Column                                                                Non-Null Count  Dtype         
---  ------                                                                --------------  -----         
 0   timestamp                                                             55 non-null     datetime64[ns]
 1   name                                                                  7 non-null      object        
 2   email                                                                 79 non-null     object        
 3   gender                                                                89 non-null     object        
 4   current_degree                                                        89 non-null     object        
 5   current_major                                                         89 non-null     object        
 6   previous_degree                             

In [91]:
temp=pd.DataFrame(columns=['current_degree','previous_degree','previous_degree_instructional_method','gender','course','instructional_method','los','grade'])

for i in range(1,4) :
    t=df.loc[:,['current_degree','previous_degree','previous_degree_instructional_method','gender',f'course_{i}_name',f'instructional_method_for_course_{i}',f'level_of_satisfaction_for_course_{i}',f'grade_for_course_{i}']]
    t.columns=['current_degree','previous_degree','previous_degree_instructional_method','gender','course','instructional_method','los','grade']
    temp=temp.append(t)
temp=temp.dropna()                   
temp=temp.reset_index(drop=True)     
temp.grade=temp.grade.apply(lambda s : 'B' if s in ('C','D') else s)
temp=temp.dropna()

In [92]:
temp.shape

(232, 8)

In [93]:
temp.instructional_method.value_counts()

Online      108
Inperson     82
Both         42
Name: instructional_method, dtype: int64

In [94]:
r1_df=temp.copy()

In [81]:
temp['r2_indicator']=''
indicator=[]
for i,each in temp.iterrows() :
    if ((each['previous_degree']=="Bachelor's") or (each['previous_degree']=="Master's"))\
    and (each['previous_degree_instructional_method']=='offline') and (each['instructional_method']=='Online'):
        indicator.append('class1')
    elif (each['current_degree']=="Bachelor's") and ((each['instructional_method']=='Online') | (each['instructional_method']=='Both')):
        indicator.append('class2')
    else :
        indicator.append('r2_NA')
temp['r2_indicator']=indicator


In [65]:
temp.r2_indicator.value_counts()

r2_NA     178
class1     52
class2      2
Name: r2_indicator, dtype: int64

In [82]:
r2_df=temp.loc[~(temp.r2_indicator=='r2_NA')]

In [83]:
pd.crosstab(r2_df.r2_indicator,r2_df.grade)

grade,A,B
r2_indicator,,
class1,40,12
class2,1,1


In [84]:
from scipy.stats import chi2_contingency
  
# defining the table
data = pd.crosstab(r2_df.r2_indicator,r2_df.grade).values
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.9751007561032065
Independent (H0 holds true)


In [95]:
r1_df

,current_degree,previous_degree,previous_degree_instructional_method,gender,course,instructional_method,los,grade
0,Master's,Bachelor's,offline,Male,Software Engineering,Online,Satisfied,A
1,Master's,Bachelor's,online,Female,Foundations of software engineering,Online,Satisfied,A
2,Master's,Bachelor's,offline,Female,Software Engineering,Both,Satisfied,A
3,Master's,Master's,online,Female,Advance machine learning,Online,Satisfied,A
4,Master's,Bachelor's,offline,Male,Human factors engineering,Inperson,Satisfied,B
...,...,...,...,...,...,...,...,...
227,Master's,Bachelor's,online,Female,Machine Learning,Inperson,Satisfied,A
228,Master's,Bachelor's,online,Male,Foundation of Software Engineering,Online,Satisfied,A
229,Master's,Bachelor's,online,Male,Project Management,Online,Satisfied,A
230,Master's,Bachelor's,offline,Male,Deep Learning,Online,Satisfied,A


In [103]:
r1_tab=pd.crosstab(r1_df.instructional_method,r1_df.grade)

In [104]:
r1_tab

grade,A,B
instructional_method,,
Both,34,8
Inperson,56,26
Online,87,21


In [105]:
r1_tab['im_total']=r1_tab.A+r1_tab.B

In [106]:
r1_tab

grade,A,B,im_total
instructional_method,,,
Both,34,8,42
Inperson,56,26,82
Online,87,21,108


In [107]:
r1_tab['A']=(r1_tab['A']/r1_tab['im_total'])*100
r1_tab['B']=(r1_tab['B']/r1_tab['im_total'])*100

In [108]:
r1_tab

grade,A,B,im_total
instructional_method,,,
Both,80.952381,19.047619,42
Inperson,68.292683,31.707317,82
Online,80.555556,19.444444,108


In [109]:
from scipy.stats import chi2_contingency
  
# defining the table
data = pd.crosstab(r1_df.instructional_method,r1_df.grade).values
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.1058812404058686
Independent (H0 holds true)


<h2>R4 : The fourth research objective is to find out whether commute
time and student’s mental and emotional state impact the learning
capability and affect the students’ performance.</h2>

In [192]:
# df.info()

In [135]:
r4_df=df.iloc[:,[8,11,12,13,16,17,18,21,22,23,26,27]].copy()

In [136]:
r4_df

,previous_major,instructional_method_for_course_1,level_of_satisfaction_for_course_1,grade_for_course_1,instructional_method_for_course_2,level_of_satisfaction_for_course_2,grade_for_course_2,instructional_method_for_course_3,level_of_satisfaction_for_course_3,grade_for_course_3,preferred_method_for_attending_classes,is_commute_time_a_factor_effecting_the_above_answer?
0,Computer Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inperson,No
1,Computer Science,Online,Satisfied,A,Inperson,Satisfied,A,Inperson,Satisfied,A,Inperson,No
2,Electronics and communication engineering,Online,Satisfied,A,Inperson,Satisfied,A,Inperson,Satisfied,A,Inperson,Maybe
3,Information Technology,Both,Satisfied,A,Inperson,Satisfied,A,Both,Satisfied,A,Online,Yes
4,NaN,Online,Satisfied,A,Online,Satisfied,A,Online,Satisfied,A,Online,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
84,Information Technology,Online,Satisfied,A,Online,Satisfied,A,Inperson,Satisfied,A,Online,Yes
85,Information Technology,Inperson,Satisfied,A,Both,Satisfied,B,Online,Satisfied,A,Inperson,No
86,Computer Science,Online,Satisfied,A,Online,Satisfied,A,Online,Satisfied,A,Online,Yes
87,Information Technology,Online,Not Satisfied,B,Online,Satisfied,B,Online,Satisfied,A,Online,Yes


In [137]:
r4_df['grade_for_course_1'].value_counts()

A    60
B    23
D     2
C     1
Name: grade_for_course_1, dtype: int64

In [138]:
r4_df.grade_for_course_1=r4_df.grade_for_course_1.apply(lambda s : 'B' if s!='A' else s)
r4_df.grade_for_course_2=r4_df.grade_for_course_2.apply(lambda s : 'B' if s!='A' else s)
r4_df.grade_for_course_3=r4_df.grade_for_course_3.apply(lambda s : 'B' if s!='A' else s)

In [139]:
r4_df['grade_for_course_1'].value_counts()

A    60
B    29
Name: grade_for_course_1, dtype: int64

In [140]:
r4_df['grade_for_course_2'].value_counts()

A    57
B    32
Name: grade_for_course_2, dtype: int64

In [141]:
r4_df['grade_for_course_3'].value_counts()

A    66
B    23
Name: grade_for_course_3, dtype: int64

In [142]:
r4_df['is_commute_time_a_factor_effecting_the_above_answer?'].value_counts()

Yes      50
No       18
Maybe    17
Name: is_commute_time_a_factor_effecting_the_above_answer?, dtype: int64

In [143]:
r4_df['is_commute_time_a_factor_effecting_the_above_answer?']=\
r4_df['is_commute_time_a_factor_effecting_the_above_answer?'].apply(lambda s : 'No' if s=='Maybe' else s)

In [144]:
r4_df['is_commute_time_a_factor_effecting_the_above_answer?'].value_counts()

Yes    50
No     35
Name: is_commute_time_a_factor_effecting_the_above_answer?, dtype: int64

In [145]:
r4_df['level_of_satisfaction_for_course_3'].value_counts()

Satisfied        71
Not Satisfied     9
Name: level_of_satisfaction_for_course_3, dtype: int64

In [133]:
#chi square test

In [155]:
samp=[]
for i in range(1,4) :
    for ind,row in r4_df.loc[:,[f'level_of_satisfaction_for_course_{i}',f'grade_for_course_{i}','is_commute_time_a_factor_effecting_the_above_answer?']].iterrows() :
        samp.append([row[f'level_of_satisfaction_for_course_{i}'],\
                     row[f'grade_for_course_{i}'],\
                    row['is_commute_time_a_factor_effecting_the_above_answer?']])
        
        
        

In [156]:
samp=pd.DataFrame(samp,columns=['los','grade','commute_flag'])

In [157]:
samp.sample(2)

,los,grade,commute_flag
198,Satisfied,A,No
15,Satisfied,A,Yes


In [158]:
samp.grade.value_counts()

A    183
B     84
Name: grade, dtype: int64

In [159]:
pd.crosstab(samp['los'],samp['grade'])

grade,A,B
los,,
Not Satisfied,6,21
Satisfied,177,46


In [160]:
pd.crosstab(samp['commute_flag'],samp['grade'])

grade,A,B
commute_flag,,
No,70,35
Yes,111,39


In [151]:
from scipy.stats import chi2_contingency
  
# defining the table
data = pd.crosstab(samp.los,samp.grade).values
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 1.046107248910522e-09
Dependent (reject H0)


In [161]:
from scipy.stats import chi2_contingency
  
# defining the table
data = pd.crosstab(samp.commute_flag,samp.grade).values
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.25860993152286826
Independent (H0 holds true)


In [136]:
# from scipy.stats import chi2_contingency
  
# # defining the table
# data = pd.crosstab(df['Current_Degree'],df['Previous_Degree']).values
# stat, p, dof, expected = chi2_contingency(data)
  
# # interpret p-value
# alpha = 0.05
# print("p value is " + str(p))
# if p <= alpha:
#     print('Dependent (reject H0)')
# else:
#     print('Independent (H0 holds true)')

In [189]:
# df.info()

<h4>R3 : The third research objective is to understand the differ-
ences between a student doing an online class versus traditional

learning in afield different from software engineering and software
engineering.</h4>

In [163]:
r3_df=df.copy()

In [164]:
temp=pd.DataFrame(columns=['gender','course','instructional_method','los','grade'])

for i in range(1,4) :
    t=df.loc[:,['gender',f'course_{i}_name',f'instructional_method_for_course_{i}',f'level_of_satisfaction_for_course_{i}',f'grade_for_course_{i}']]
    t.columns=['gender','course','instructional_method','los','grade']
    temp=temp.append(t)
                     
temp=temp.reset_index(drop=True)     

temp.grade=temp.grade.apply(lambda s : 'B' if s=='D' else s)
temp=temp.dropna()
                 

In [190]:
# temp.info()

In [166]:
temp

,gender,course,instructional_method,los,grade
1,Male,Software Engineering,Online,Satisfied,A
2,Female,Foundations of software engineering,Online,Satisfied,A
3,Female,Software Engineering,Both,Satisfied,A
4,Female,Advance machine learning,Online,Satisfied,A
5,Male,Human factors engineering,Inperson,Satisfied,B
...,...,...,...,...,...
262,Female,Machine Learning,Inperson,Satisfied,A
263,Male,Foundation of Software Engineering,Online,Satisfied,A
264,Male,Project Management,Online,Satisfied,A
265,Male,Deep Learning,Online,Satisfied,A


In [167]:
temp.gender.value_counts()

Male      179
Female     67
Name: gender, dtype: int64

In [171]:
temp.grade=temp.grade.apply(lambda s : 'B' if s!='A' else s)
temp.grade.value_counts()

A    181
B     65
Name: grade, dtype: int64

In [172]:
temp.los.value_counts()

Satisfied        219
Not Satisfied     27
Name: los, dtype: int64

In [173]:
temp.instructional_method.value_counts()

Online      111
Inperson     88
Both         47
Name: instructional_method, dtype: int64

In [175]:
temp['course_label']='kavy'

In [191]:
# temp.loc[temp.course.str.lower().str.contains('software engineering')]

In [177]:
temp['course_label']=temp.course.apply(lambda s : 'soft' if 'software engineering' in s.lower() else 'hard')

In [178]:
temp

,gender,course,instructional_method,los,grade,course_label
1,Male,Software Engineering,Online,Satisfied,A,soft
2,Female,Foundations of software engineering,Online,Satisfied,A,soft
3,Female,Software Engineering,Both,Satisfied,A,soft
4,Female,Advance machine learning,Online,Satisfied,A,hard
5,Male,Human factors engineering,Inperson,Satisfied,B,hard
...,...,...,...,...,...,...
262,Female,Machine Learning,Inperson,Satisfied,A,hard
263,Male,Foundation of Software Engineering,Online,Satisfied,A,soft
264,Male,Project Management,Online,Satisfied,A,hard
265,Male,Deep Learning,Online,Satisfied,A,hard


In [179]:
temp.instructional_method.unique()

array(['Online', 'Both', 'Inperson'], dtype=object)

In [180]:
temp.course_label.unique()

array(['soft', 'hard'], dtype=object)

In [181]:
print()
for each in ['Online', 'Both', 'Inperson'] :
    temp_df=temp.loc[temp['instructional_method']==each]
    print(each)
    print(pd.crosstab(temp_df.grade,temp_df.course_label))
    print()


Online
course_label  hard  soft
grade                   
A               63    24
B               23     1

Both
course_label  hard  soft
grade                   
A               35     2
B               10     0

Inperson
course_label  hard  soft
grade                   
A               56     1
B               31     0



In [182]:
print()
for each in ['soft', 'hard'] :
    temp_df=temp.loc[temp['course_label']==each]
    print(each)
    print(pd.crosstab(temp_df.grade,temp_df.instructional_method))
    print()


soft
instructional_method  Both  Inperson  Online
grade                                       
A                        2         1      24
B                        0         0       1

hard
instructional_method  Both  Inperson  Online
grade                                       
A                       35        56      63
B                       10        31      23



In [186]:
from scipy.stats import chi2_contingency
  
# defining the table
data = pd.crosstab(temp.course_label,temp.grade).values
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.007241255596707472
Dependent (reject H0)


In [187]:
from scipy.stats import chi2_contingency
  
# defining the table
data = pd.crosstab(temp.course_label,temp.gender).values
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.0019362192311904068
Dependent (reject H0)


In [188]:
from scipy.stats import chi2_contingency
  
# defining the table
data = pd.crosstab(temp.course_label,temp.los).values
stat, p, dof, expected = chi2_contingency(data)
  
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.31233268033864947
Independent (H0 holds true)
